In [ ]:
#These lines set up inline plotting, and apply a standard size
%matplotlib inline
import matplotlib
matplotlib.rc('font', **{'size': 22})
# Standard includes
import datetime
import re
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import matplotlib.pyplot as plt
from time import time

In [ ]:
# Define the query
my_query = {
  "size": 0,
  "query": {
    "filtered": {
      "query": { "match_all": {} },
      "filter": {
        "bool": {
          "must": [
            {
              "range": {
                "@timestamp": {
                  "gte": 1451602800000,
                  "lte": 1475618400000,
                  "format": "epoch_millis"
                }
              }
            },
            {"term": {"event_type": "transfer-failed"}},
            {"term": {"payload.scope": "data16_13TeV"}}
          ],
          "must_not": []
        }
      }
    }
  }
}

In [ ]:
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)
my_index = "rucio-events*"
scroll = scan(es, query=my_query, index=my_index, scroll='5m', timeout="5m", size=100)

In [ ]:
count=0
t0 = time()
allSizes = []
endpointPairs = {}
for res in scroll:
    if not count%1000:  print(count)
    if count<1: print(res)
    count += 1
    if res['_source']['event_type']=='transfer-failed':
        allSizes.append(res['_source']['payload']['file-size'])
        source = res['_source']['payload']['src-rse']
        destination = res['_source']['payload']['dst-rse']
        pair = (source,destination)
        if pair not in endpointPairs.keys():
            endpointPairs[pair] = 1
        if pair in endpointPairs.keys():
            endpointPairs[pair] = endpointPairs[pair]+1
t1 = time()

In [ ]:
print (count, 'events in %f seconds.' %(t1-t0), count/(t1-t0), "ev/s.")

In [ ]:
import operator
sorted_endpointPairs = sorted(endpointPairs.items(), key=operator.itemgetter(1))
for item in sorted_endpointPairs:
    print(item[0],'\t\t',item[1])